In [112]:
# load packages 
library(pacman)
p_load(
    tidyverse, data.table, dtplyr, reshape2, 
    archive, kableExtra, SPARQL, janitor, 
    httr, jsonlite)
# set option
options(dplyr.summarise.inform = FALSE)
gray_scale <- c('#F3F4F8','#D2D4DA', '#B3B5BD', 
                '#9496A1', '#7d7f89', '#777986', 
                '#656673', '#5B5D6B', '#4d505e',
                '#404352', '#2b2d3b', '#282A3A',
                '#1b1c2a', '#191a2b',
                '#141626', '#101223')

In [2]:
han_names <- fread('work/notebooks/patent/data/202208_HAN_NAMES.txt')

In [7]:
dim(han_names)

[1] 4191007       3

In [7]:
head(han_names)

HAN_ID,Clean_name,Person_ctry_code
<int>,<chr>,<chr>
1,& HAMBOURG NIENDORF,DE
2,& KK,JP
3,“ASTRONIT” CLOSE CORP,RU
4,“DEUTSCHE SEE” GMBH,DE
5,“EFIRNOIE” OPEN JOINT STOCK CO,RU
6,“EUROSTANDART” LTD LIABILITY CO,RU


In [3]:
de_firms <- fread('work/notebooks/patent/data/orbis_de_matched_l.csv')

In [55]:
han_names %>%
    .[1:5,] %>%
    kable('pipe', align='ccc')



| HAN_ID |           Clean_name           | Person_ctry_code |
|:------:|:------------------------------:|:----------------:|
|   1    |      & HAMBOURG NIENDORF       |        DE        |
|   2    |              & KK              |        JP        |
|   3    |     “ASTRONIT” CLOSE CORP      |        RU        |
|   4    |      “DEUTSCHE SEE” GMBH       |        DE        |
|   5    | “EFIRNOIE” OPEN JOINT STOCK CO |        RU        |

In [53]:
de_firms %>%
    .[1:5, 2:3]

name_native,name_internat
<chr>,<chr>
Airbus Defence and Space GmbH,Airbus Defence and Space GmbH
EurA AG,EurA AG
TuTech Innovation GmbH,TuTech Innovation GmbH
FFT Produktionssysteme GmbH & Co. KG.,FFT Produktionssysteme GmbH & Co. KG.
Diehl Aviation Laupheim GmbH,Diehl Aviation Laupheim GmbH


In [83]:
# query
airbus <- toupper('Airbus Defence')
han_names %>%
    .[Person_ctry_code == 'DE'] %>%
    .[Clean_name %like% airbus] %>%
    .[,HAN_ID] -> airbus_han_ids
airbus_han_ids

[1]   60513   62422 3637004 4401227 4527012

In [61]:
han_patents <- fread('work/notebooks/patent/data/202208_HAN_PATENTS.txt')

In [86]:
han_patents %>%
    .[HAN_ID %in% airbus_han_ids] %>%
    .[, .N, by=Publn_auth] -> foo
    transform(adorn_totals(foo)) %>%
    transpose() %>%
    row_to_names(row_number=1)

,EP,US,WO,Total
,<chr>,<chr>,<chr>,<chr>
2,716,415,88,1219


In [84]:
han_patents %>%
    .[HAN_ID %in% airbus_han_ids] %>%
    .[sample(.N, 5)]

HAN_ID,HARM_ID,Appln_id,Publn_auth,Patent_number
<int>,<int>,<int>,<chr>,<chr>
60513,60513,442096647,EP,EP3112597
60513,60513,417401751,EP,EP2913271
60513,60513,404985890,EP,EP2825450
60513,60513,527283926,US,US2020070202
60513,60513,544408405,WO,WO2021123759


In [96]:
han_patents %>%
    .[HAN_ID %in% airbus_han_ids] %>%
    .[Publn_auth == 'EP'] -> airbus_ep_patents
    head(airbus_ep_patents)

HAN_ID,HARM_ID,Appln_id,Publn_auth,Patent_number
<int>,<int>,<int>,<chr>,<chr>
60513,60513,213,EP,EP2030891
60513,60513,65448,EP,EP2025928
60513,60513,156990,EP,EP1920908
60513,60513,161551,EP,EP1972896
60513,60513,173385,EP,EP2134522
60513,60513,173386,EP,EP2136979


In [111]:
res <- get('"httr"')

$format
[1] "linked-data-api"

$version
[1] "0.2"

$result
$result$`_about`
[1] "http://data.epo.org/linked-data/data/publication/EP/1972896.json"

$result$definition
[1] "http://data.epo.org/linked-data/meta/data/publication/_st3Code/_publicationNumber.json"

$result$extendedMetadataVersion
[1] "http://data.epo.org/linked-data/data/publication/EP/1972896.json?_metadata=all"

$result$first
[1] "http://data.epo.org/linked-data/data/publication/EP/1972896.json?_page=0"

$result$hasPart
[1] "http://data.epo.org/linked-data/data/publication/EP/1972896.json"

$result$isPartOf
[1] "http://data.epo.org/linked-data/data/publication/EP/1972896.json"

$result$items
$result$items[[1]]
$result$items[[1]]$`_about`
[1] "http://data.epo.org/linked-data/data/publication/EP/1972896/A2/-"

$result$items[[1]]$application
$result$items[[1]]$application$`_about`
[1] "http://data.epo.org/linked-data/id/application/EP/08004318"

$result$items[[1]]$application$applicationNumber
[1] "08004318"


$result$items[[1]]$label
[1] "EP 1972896 A2"

$result$items[[1]]$publicationAuthority
[1] "http://data.epo.org/linked-data/id/st3/EP"

$result$items[[1]]$publicationDate
[1] "Wed, 24 Sep 2008"

$result$items[[1]]$publicationKind
$result$items[[1]]$publicationKind$`_about`
[1] "http://data.epo.org/linked-data/def/patent/publicationKind_A2"

$result$items[[1]]$publicationKind$label
[1] "A2"



$result$items[[2]]
$result$items[[2]]$`_about`
[1] "http://data.epo.org/linked-data/data/publication/EP/1972896/A3/-"

$result$items[[2]]$application
$result$items[[2]]$application$`_about`
[1] "http://data.epo.org/linked-data/id/application/EP/08004318"

$result$items[[2]]$application$applicationNumber
[1] "08004318"


$result$items[[2]]$label
[1] "EP 1972896 A3"

$result$items[[2]]$publicationAuthority
[1] "http://data.epo.org/linked-data/id/st3/EP"

$result$items[[2]]$publicationDate
[1] "Wed, 07 Nov 2012"

$result$items[[2]]$publicationKind
$result$items[[2]]$publicationKind$`_about`
[1] "http://data.epo.org/linked-data/def/patent/publicationKind_A3"

$result$items[[2]]$publicationKind$label
[1] "A3"



$result$items[[3]]
$result$items[[3]]$`_about`
[1] "http://data.epo.org/linked-data/data/publication/EP/1972896/B1/-"

$result$items[[3]]$application
$result$items[[3]]$application$`_about`
[1] "http://data.epo.org/linked-data/id/application/EP/08004318"

$result$items[[3]]$application$applicationNumber
[1] "08004318"


$result$items[[3]]$label
[1] "EP 1972896 B1"

$result$items[[3]]$publicationAuthority
[1] "http://data.epo.org/linked-data/id/st3/EP"

$result$items[[3]]$publicationDate
[1] "Wed, 06 May 2015"

$result$items[[3]]$publicationKind
$result$items[[3]]$publicationKind$`_about`
[1] "http://data.epo.org/linked-data/def/patent/publicationKind_B1"

$result$items[[3]]$publicationKind$label
[1] "B1"




$result$itemsPerPage
[1] 10

$result$notice
$result$notice$label
[1] "Provided by the European Patent Office."


$result$page
[1] 0

$result$startIndex
[1] 1

$result$type
$result$type[[1]]
[1] "http://purl.org/linked-data/api/vocab#ListEndpoint"

$result$type[[2]]
[1] "http://purl.org/linked-data/api/vocab#Page"